In [1]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 
from pandas import ExcelWriter
from pandas import ExcelFile
import time

In [2]:
def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


In [3]:
def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists



In [4]:
# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0

In [5]:
# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



In [6]:
# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1



In [7]:
# In[7]:

def crawl(original_url, num_Id, output_file):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = [original_url]
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()

    Di={}

    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
            
            for child in child_list:
                if child != None:
                    ch=child
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1 and ch not in Di:
                        child_list_filtered.append(child)
                        Di[ch]=1
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    df.to_csv(output_file + '/' + str(num_Id) + '_' + str(url_to_check) +  '.csv', sep=',', encoding='utf-8')
    return df, num_Id

In [8]:
import requests

page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u", ".join(t.strip() for t in visible_texts)
web_text = []

In [14]:
import json
import urllib
import os 
extensions=('.png', '.jpg','.jpeg','.pdf', '.gif','.ico')
def scrape(df, scraped_website_path, website_index, website_name):
    TempD={}
    for index,item in df.iterrows():
        print(item[1],item[0])
        try:      
            #print(item[1])
            if item[1].endswith(extensions):
                print(item[1],"skipped")
                pass
            
            else:
                print(item[1],"printed")
                html = urllib.request.urlopen(item[1]).read()
                #web_text.append(text_from_html(html))
                print(text_from_html(html))
                time.sleep(3)
                #save_url(str(item[1]), index)
                TempD[str(item[1])]=text_from_html(html)
        except: 
            pass
    with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
        json.dump(TempD, outfile)
        
    return 1

In [15]:
scrape1("http://www.hscast.com//quality")

NameError: name 'scrape1' is not defined

In [16]:

# web_crawler_all() = Scraper for multiple websites# web_cr 
'''
Inputs
-----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
web_col = the column no of in which the website hompages are located 
index_col = the column with indices of websites
parent_col = the column with the parent names
output_folder = path of the output folder where we wish to save the websites 
from_website = the index no. from website (typically starting from 1)
to_website = the index no of the website till we want to continue scraping (scraped websites will include the 
                websites with this index)

Output 
-----------
CSV files containing a list of child links for each website are saved into the output_folder path 

'''
def web_crawler_all(excel_path, sheet, web_col, index_col, parent_col,  output_folder, from_website, to_website):
    df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]
    print(df)
    for index,row in df.iterrows():
        num_Id=int(row[index_col])
        original_url=row[web_col]
        print("Website no : " + str(num_Id))
        print("Website name : " + str(original_url))
        
        try:
            websites= pd.read_csv(output_folder + str(int(num_Id)) + '_' + str(original_url) + '.csv')
            print("website is already crawled")
        except:
            websites, num_Id = crawl(original_url ,num_Id ,output_folder)
            
        
    print("All links from the websites indicated have been crawled")
    print("Output File :  " + str(output_folder))
    print("No. of websites scraped : " + str(to_website - from_website + 1))
    return 1

### Things to be added
- index to track what has been scrolled 
- time taken by the href function
- file structure for all webpages


In [17]:
'''
Inputs 
----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
crawled_website_path = path of file where all the crawled websites are stored  
scraped_website_path = path of file where all the scraped websites are stored
from_website - to_website = website index number from-to which we want to scrape and store in JSON 
web_col = column number of websites (0,1,2....)

Output
----------
JSON files for each parent website in scraped_website_path folder. The path of the scraped websites 
is the same as that crawled websites 
'''

def web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col):
    #get excel sheet containing websites 
  #  df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]   
    
    #loop over all the websites and extract child links. Store them in JSON files 
    
    for index, rows in df.iterrows():
        #setting the website index and name 
        website_index = int(rows[0] )
        website_name = rows[web_col]
        
        #generating filename and getting the csv file of child links 
        filename = crawled_website_path + '/'+ str(website_index)+ '_' + str(get_base_url(website_name)) + ".csv"
        df_of_child_links = pd.read_csv(filename)
        print(filename,df_of_child_links )
        #executing the scrape function 
        flag = scrape(df_of_child_links , scraped_website_path, website_index, website_name) 
        
        #checkpoint to see if scraping is complete 
        if flag == 1 :
            print(website_name + ' has been scraped successfully')
            #print('scraping percentage complete = ' + str((i/to_website)*100) + '%')
            flag = 0

In [ ]:
#executing the scraper function 

excel_path = 'D:\WHIN/trial1.xlsx'
sheet = 'From List of Companies'
df = pd.read_excel(excel_path , sheet_name=sheet)
crawled_website_path = 'D:\WHIN\Scraped_data/crawled_websites/'
scraped_website_path = 'D:\WHIN\Scraped_data\scraped_websites/'
from_website = 1
to_website = 4
web_col = 2 
df = pd.read_excel(excel_path , sheet_name=sheet)
#print(df)
web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col)

D:\WHIN\Scraped_data/crawled_websites//2_www.carmeleng.com.csv     Unnamed: 0                                                  0  \
0            0            http://www.carmeleng.com/contact_us.asp   
1            1            http://www.carmeleng.com/newsletter.asp   
2            2  http://www.carmeleng.com/http:/www.gobuckaroo.com   
3            3               http://www.carmeleng.com/default.asp   
4            4              http://www.carmeleng.com//favicon.ico   
5            5           http://www.carmeleng.com/carmelstyle.css   
6            6  http://www.carmeleng.com/images/CE_SplashPage-...   
7            7  http://www.carmeleng.com/images/CE_SplashPage-...   
8            8  http://www.carmeleng.com/images/CE_SplashPage-...   
9            9          http://www.carmeleng.com/images/white.gif   
10          10  http://www.carmeleng.com/images/CE_SplashPage-...   
11          11  http://www.carmeleng.com/images/CE_SplashPage-...   
12          12  http://www.carmeleng.com

http://www.carmeleng.com//favicon.ico 4
http://www.carmeleng.com//favicon.ico skipped
http://www.carmeleng.com/carmelstyle.css 5
http://www.carmeleng.com/carmelstyle.css printed

http://www.carmeleng.com/images/CE_SplashPage-2_03.jpg 6
http://www.carmeleng.com/images/CE_SplashPage-2_03.jpg skipped
http://www.carmeleng.com/images/CE_SplashPage-2_NEW_16.jpg 7
http://www.carmeleng.com/images/CE_SplashPage-2_NEW_16.jpg skipped
http://www.carmeleng.com/images/CE_SplashPage-2_25.jpg 8
http://www.carmeleng.com/images/CE_SplashPage-2_25.jpg skipped
http://www.carmeleng.com/images/white.gif 9
http://www.carmeleng.com/images/white.gif skipped
http://www.carmeleng.com/images/CE_SplashPage-2_NEW_18.jpg 10
http://www.carmeleng.com/images/CE_SplashPage-2_NEW_18.jpg skipped
http://www.carmeleng.com/images/CE_SplashPage-2_22.jpg 11
http://www.carmeleng.com/images/CE_SplashPage-2_22.jpg skipped
http://www.carmeleng.com/images/CE_SplashPage-2_30.jpg 12
http://www.carmeleng.com/images/CE_SplashPage-2_30.

, , , , , , , , , , Site Map, , Site Credits, , Home, , , , About, , News, , Products, , Service, , Contact, , , , , , , , , Site Map, , , AC Locomotives Categories | Dayton-Phoenix Group, , Information about the product categories offered by Dayton-Phoenix Group, , Air Products and Timers Categories | Dayton-Phoenix Group, , Information about the product categories offered by Dayton-Phoenix Group, , All Product Categories | Dayton-Phoenix Group, , Information about the product categories offered by Dayton-Phoenix Group, , Auxiliary Generators Categories | Dayton-Phoenix Group, , Information about the product categories offered by Dayton-Phoenix Group, , Awnings Categories | Dayton-Phoenix Group, , Information about the product categories offered by Dayton-Phoenix Group, , Brake Grid Fan Assemblies Categories | Dayton-Phoenix Group, , Information about the product categories offered by Dayton-Phoenix Group, , Cab Cooling & Heating Categories | Dayton-Phoenix Group, , Information about 

https://www.dayton-phoenix.com/siteCredits.php 1
https://www.dayton-phoenix.com/siteCredits.php printed
, , , , , , , , , , Site Map, , Site Credits, , Home, , , , About, , News, , Products, , Service, , Contact, , , , , , , Technical Resources, , , Debian GNU/Linux, , Apache HTTP Server, , PHP Scripting Language, , MySQL Database Server, , , Standards Guidelines, , , Cascading Style Sheets, , XHTML 1.0, , Microformats, , , , , Site Credits, , About This Website, , Dayton-Phoenix Group is committed to making dayton-phoenix.com a valuable resource for our customers. To that end, this website has been built using the latest web standards allowing for the widest audience possible. We hope that you find it usable., , Feedback, , We are always interested in feedback regarding this website. Please direct any comments or concerns to, webmaster@dayton-phoenix.com, ., , , , , , , , , , , , , , , © 2018 Dayton-Phoenix Group | All Rights Reserved, , , , , , , , , 
https://www.dayton-phoenix.com/d

, , , , , , , , , , Site Map, , Site Credits, , Home, , , , About, , News, , Products, , Service, , Contact, , , , , , , Service, , , Service Centers, , Warranty, , , , , Service Centers, , , Dayton-Phoenix offers service at any of our, Service Centers, listed below., , Please contact our Service Department at 800-657-0707 or, service@dayton-phoenix.com, ., , , Clima Center Ar Condicionado, , mkt@climacenter.srv.br, , , Avenue Vereador Laudelino Schetino, Galpão 14, , Democrata, ,, Juiz de Fora/MG, ,, 36035-205, , Brazil, , , tel: 011-55-32-3232-1100, , http://www.climacenter.srv.br, , , , DPG Service: Dayton, , , 1619 Kuntz Rd., , Dayton, ,, OH, ,, 45404-1240, , USA, , , , , DPG Service: Gothenburg, , , 1020 4th Street, , Gothenburg, ,, NE, ,, 69138-2266, , USA, , , , , DPG Service: Houston, , , 3340 B Greens Road Suite 350, , Houston, ,, TX, ,, 77032, , USA, , , , , McClure Mechanical Services, , , 4101 North Sixth Street, , Harrisburg, ,, PA, ,, 17105-2630, , USA, , , , , Mid Nebras

...

https://www.oerlikon.com/balzers/tr/tr/ 153
https://www.oerlikon.com/balzers/tr/tr/ printed
, , , , , , , , , , , , , , , , , \uf1ad, , , , , , , , , Portföy, Kaplama hizmetleri, Balzers yüzey çözümleri, Equipment, Surface Technologies, Uygulamalar, Tools, Components, Pazarlar, Automotive and Transportation, Aerospace, Medical, Food & Packaging, Energy, Consumer Goods, Engineering, Balzers Tutkusu, The Balzers Story, Responsibility, Career, Power of Innovation, Social Web, References, Müşteri Dergisi, Hizmet Alanı, Genel Merkez İletişim Bilgileri, Events, İstek formu, Belgeleri indir, Press, Dünya Geneli, Şirketimiz, Career, Competencies, Tarihçe, Management, Dünya Genelinde Kaplama Servisi Merkezlerimiz, Oerlikon Balzers Türkiye, , , , , , , , Grup, , , , Kurumsal, , , , Yüzey Çözümleri Dilimi, , , Balzers, , Metco, , Additive Manufacturing, , , , , Yapay Elyaf Dilimi, , , Barmag, , Neumag, , , , , Tahrik Sistemleri Dilimi, , , Graziano, , Fairfield, , , , , , , , Ülkeler, , , Global,

https://www.oerlikon.com/balzers/uk/en/ 154
https://www.oerlikon.com/balzers/uk/en/ printed
, , , , , , , , , , , , , , , , , \uf1ad, , , , , , , , , Portfolio, Job coating services, Balzers surface solutions, Equipment, Surface technologies, Applications, Tools, Components, Markets, Automotive and Transportation, Aerospace, Medical, Food & Packaging, Energy, Consumer Goods, Engineering, Balzers Passion, The Balzers Story, Responsibility, Career, Power of Innovation, Social Web, References, Customer Magazine, Service Area, Contacts, Events, Request form, Downloads Documents, Press, Worldwide, Our Company, Career, Competencies, History, Management, Management United Kingdom, Worldwide locations, Locations United Kingdom, , , , , , , , Group, , , , Corporate, , , , Surface Solutions Segment, , , Balzers, , Metco, , Additive Manufacturing, , , , , Manmade Fibers Segment, , , Barmag, , Neumag, , , , , Drive Systems Segment, , , Graziano, , Fairfield, , , , , , , , Countries, , , Global, He

https://www.oerlikon.com/balzers/cn/cn/ 155
https://www.oerlikon.com/balzers/cn/cn/ printed
, , , , , , , , , , , , , , , , , \uf1ad, , , , , , , , , 产品组合, 涂层服务, 巴尔查斯表面处理解决方案, 设备, 表面处理技术, 应用, 工具, 零部件, 市场, 汽车与交通运输, 航空航天, 医疗、食品与包装, 能源, 消费品, 工程, 巴尔查斯的激情, 欧瑞康巴尔查斯的历史, 责任, 工作机会, 创新的力量, 社交网络, 参考文献, 客户杂志, 服务, 联系方式, 活动, 问询表格, 下载文档, 新闻, 全球, 关于我们, 工作机会, 能力, 历史, 管理, 遍布世界的涂层中心, 中国区地址, , , , , , , , 集团, , , , 企业, , , , 表面处理事业板块, , , 欧瑞康巴尔查斯, , 欧瑞康美科, , 增材制造, , , , , 化学纤维事业板块, , , 巴马格, , 纽马格, , , , , 传动系统事业板块, , , 古加诺, , 菲尔费德, , , , , , , , 各国网页, , , Global, Headerquarters, English, German, Americas, Argentina, Spanish, Brazil, Portuguese, Canada, English, Mexiko, Spanish, USA, English, Europe, Austria, German, English, Belgium, English, Czech Republic, Czech, Finland, Finnish, English, France, French, Germany, German, Hungary, Hungarian, Italy, Italian, Liechtenstein, German, French, Luxembourg, English, Netherlands, English, Poland, Polish, Portugal, Portuguese, Romania, Romanian, Russia, Russian, 

https://www.oerlikon.com/balzers/jp/ja/ 157
https://www.oerlikon.com/balzers/jp/ja/ printed
, , , , , , , , , , , , , , , , , \uf1ad, , , , , , , , , ポートフォリオ, コーティング受託加工, バルザースの表面処理ソリューション, 装置, 表面処理テクノロジー, アプリケーション, 工具・金型, 部品, マーケット, 自動車・輸送産業, 航空宇宙産業, 医療、食品および梱包, エネルギー, 消費者向け製品, エンジニアリング, バルザースの情熱, バルザースの歴史, 責任, キャリア, イノベーションの力, ソーシャルウェブ, リファレンス, カスタマーマガジン, サービス エリア, イベント, リクエストフォーム, ドキュメントのダウンロード, プレス, 世界の拠点, 会社情報, キャリア, コンピテンシー, 歴史, 経営陣, 世界の拠点, 日本の拠点, 日本エリコンバルザース, , , , , , , , エリコンメテコグループ, , , , 本社, , , , 表面ソリューション部門, , , バルザース, , メテコ, , Additive Manufacturing, , , , , 人工ファイバー部門, , , バルマグ, , ニューマグ, , , , , ドライブシステム部門, , , グラツィアーノ, , フェアフィールド, , , , , , , , Countries, , , Global, Headerquarters, English, German, Americas, Argentina, Spanish, Brazil, Portuguese, Canada, English, Mexiko, Spanish, USA, English, Europe, Austria, German, English, Belgium, English, Czech Republic, Czech, Finland, Finnish, English, France, French, Germany, German, Hungary, Hungarian, Italy, Italian, Liechte

https://www.oerlikon.com/balzers/my/ 159
https://www.oerlikon.com/balzers/my/ printed


In [ ]:
#executing the crawler function 

excel_path = 'D:\WHIN/trial1.xlsx'
sheet = 'From List of Companies'
web_col = 2
index_col = 0
parent_col = 1
output_folder = 'D:\WHIN\scraped_data\crawled_websites/'
from_website = 4 
to_website = 9

web_crawler_all(excel_path,sheet, web_col, index_col, parent_col,output_folder, from_website, to_website)

   Serial No.               Company Name  \
3         5.0         Oerlikon Fairfield   
4         6.0           Grand Industrial   
5         7.0    Harrison Steel Castings   
6         8.0            Hose Technology   
7         9.0  Kirby Risk Service Center   
8        10.0    Lafayette Wire Products   

                                  Website  Has a Website?\n(1=Yes, 0=No)  \
3  https://www.oerlikon.com/fairfield/en/                            NaN   
4        https://www.grandindustrial.com/                            NaN   
5                  http://www.hscast.com/                            NaN   
6                     http://hosetec.com/                            NaN   
7               https://www.kirbyrisk.com                            NaN   
8               http://lafayettewire.com/                            NaN   

  Static/Dynamic Website?  \
3                     NaN   
4                     NaN   
5                     NaN   
6                     NaN   
7            

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/portrait/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/company-profile/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/company-overview/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/organization/board-of-directors/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/key-figures/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/market-segments/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/worldwide-locations/
https://www.oerlikon.com/en/company/company-overview/surface-solutions/?dyk24=true/en/company/compliance/
https://www.oerlikon.com/en/

In [43]:
def scrape1(website_name):
    TempD={}
    #for index,item in df.iterrows():
    extensions=('.png', '.jpg','.jpeg','.pdf', '.gif','.ico')    #file_save(item[1],item[0])
    try:      
        print(website_name)
        print( website_name.endswith("extensions"))
        if 1:
              # print(item[1])
            html = urllib.request.urlopen(website_name).read()
            #print(html)
             #web_text.append(text_from_html(html))
            print(text_from_html(html))
            #    time.sleep(3)
                #save_url(str(item[1]), index)
           # TempD[str(item[1])]=text_from_html(html)
    except: 
        pass
   # with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
       # json.dump(TempD, outfile)
        
    return 1